In [1]:
import osmnx as ox
import networkx as nx

import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, sjoin
from geopandas.tools import sjoin
from shapely.geometry import LineString, Point, Polygon, MultiPolygon

import numpy as np
from sklearn.neighbors import KDTree

from tqdm import tqdm  # For progress bar
from tqdm.auto import tqdm

import threading
from queue import Queue
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import os

import json
import re

import warnings
warnings.filterwarnings("ignore")


C:\Users\Eigenaar\AppData\Local\Temp\ipykernel_49912\3832976404.py:24: DeprecationWarning: WKTReadingError is deprecated and will be removed in a future version. Use ShapelyError instead (functions previously raising {name} will now raise a ShapelyError instead).
  from shapely.errors import WKTReadingError


In [2]:
# Define the coordinates in WGS84 (EPSG:4326). The current coordinates are for a bbox roughly covering the greater region of Amsterdam
north, south, east, west = 52.43, 52.28, 5.10, 4.74

In [3]:
#import buildings from the BAG and set to epsg 28992
buildings = gpd.read_file("./root/Inputs/buildings.gpkg") 
buildings = buildings.to_crs(epsg=28992)

In [4]:
#import clusters and set to epsg 28992
destinations = gpd.read_file("./root/Outputs/1_Essential_Service_Clusters.geojson")
destinations = destinations.to_crs(epsg=28992)
destinations = destinations[destinations.geometry.type == 'Polygon']

In [5]:
#filter buildings for type, status and construction year

buildings = buildings.fillna('Unknown')
buildings = buildings[buildings['gebruiksdoel'].str.contains('woonfunctie')]
buildings = buildings[buildings['status'] == 'Pand in gebruik']
buildings = buildings[buildings['bouwjaar'] < 2023]

Fetch network

In [7]:
#fetch network
cf = """
     ["area"!~"yes"]
     ["highway"]
     ["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]
     ["foot"!~"no"]
     ["service"!~"private"]
     ["access"!~"private"]
     """

try:
    G = ox.graph_from_bbox(north, south, east, west, custom_filter=cf, network_type='walk', simplify= False, truncate_by_edge=True) 
except Exception as e:
    print(f"Error while fetching the street network: {e}")


Assign correct crs to nodes and edges

In [8]:
# Convert network to GeoDataFrame
nodes, edges = ox.graph_to_gdfs(G)
# Convert network to the same CRS as buildings
edges = edges.to_crs(buildings.crs)
nodes = nodes.to_crs(buildings.crs)

Buffer clusters with no overlay

In [9]:
# Perform the spatial join
overlapping_nodes = sjoin(destinations, nodes, how='inner', predicate='intersects')

# Select unique destination indices from the spatial join
unique_destination_indices = overlapping_nodes.index.drop_duplicates()

# Extract only unique polygons from destinations
unique_destinations = destinations.loc[unique_destination_indices]

# Add a 20-meter buffer to the non-overlapping destinations
non_overlapping_destinations = destinations.loc[~destinations.index.isin(unique_destination_indices)]
non_overlapping_destinations['geometry'] = non_overlapping_destinations.geometry.buffer(20)

# Combine the unique destinations and buffered non-overlapping destinations
destinations = GeoDataFrame(
    pd.concat([unique_destinations, non_overlapping_destinations]).drop_duplicates(subset='geometry'),
    crs=destinations.crs
)

# Reset the index to ensure consistency
destinations.reset_index(drop=True, inplace=True)

Overlay nodes with clusters

In [10]:
# Perform a spatial join
overlapping_nodes = sjoin(nodes, destinations, how='inner', predicate='intersects')

# Ensure the `network_cluster` column is transferred from `destinations` to `overlapping_nodes`
overlapping_nodes['network_cluster'] = overlapping_nodes.index_right.map(destinations['network_cluster'])

Create a KD Tree for each cluster for fast distance calculations

In [11]:
# Prepare KD-Trees for polygon nodes
kd_trees = {}

# Ensure the GeoDataFrame is sorted by 'polygon_id'
overlapping_nodes = overlapping_nodes.sort_values('network_cluster')

# Group by 'network_cluster' and build KD-Trees
for network_cluster, group in overlapping_nodes.groupby('network_cluster'):
    coordinates = np.array(list(group.geometry.apply(lambda geom: (geom.x, geom.y))))
    kd_tree = KDTree(coordinates)
    kd_trees[network_cluster] = {
        'kd_tree': kd_tree,
        'data': group
    }

Centroid calculation and find closest node to centroid

In [14]:

# Function to compute centroid and closest node for a single row
def process_row(row, nodes_geometry):
    centroid = row.geometry.centroid
    closest_node_id = nodes_geometry.distance(centroid).idxmin()
    closest_node_geometry = nodes_geometry.loc[closest_node_id]
    #print(f"\rProcessing row: {row.identificatie}", end="", flush=True)
    return row.identificatie, centroid, closest_node_id, closest_node_geometry

# Parallel processing function
def calculate_centroids_and_closest_nodes(buildings_df, nodes):
    num_threads = os.cpu_count()  # Get the number of available threads
    print(f"Using {num_threads} threads for parallel processing")
    
    nodes_geometry = nodes.geometry  # Extract node geometries once for efficiency
    results = []
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = {
            executor.submit(process_row, row, nodes_geometry): row.identificatie
            for row in buildings_df.itertuples(index=False)
        }
        
        for future in tqdm(concurrent.futures.as_completed(futures), 
                           total=len(futures), 
                           desc="Centroid and Closest Node Calculation"):
            results.append(future.result())
    
    return results
  
# Process centroids and closest nodes in parallel
centroid_and_closest_node_results = calculate_centroids_and_closest_nodes(buildings, nodes)

# Convert results back to GeoDataFrame
result_df = pd.DataFrame(
    centroid_and_closest_node_results, 
    columns=['identificatie', 'centroid', 'closest_node', 'closest_node_geometry']
)

# Convert the centroids and closest_node_geometry back to GeoSeries
result_df['centroid'] = gpd.GeoSeries(result_df['centroid'], crs="EPSG:4326")
result_df['closest_node_geometry'] = gpd.GeoSeries(result_df['closest_node_geometry'], crs="EPSG:4326")

# Merge results into the original buildings DataFrame
buildings_clean = buildings.merge(result_df, on="identificatie")

print(buildings_clean.head())

Using 12 threads for parallel processing


Centroid and Closest Node Calculation:   0%|          | 0/100 [00:00<?, ?it/s]

   feature_id                                        rdf_seealso  \
0     3699610  http://bag.basisregistraties.overheid.nl/bag/i...   
1     3827373  http://bag.basisregistraties.overheid.nl/bag/i...   
2     3850047  http://bag.basisregistraties.overheid.nl/bag/i...   
3     3757484  http://bag.basisregistraties.overheid.nl/bag/i...   
4     3797748  http://bag.basisregistraties.overheid.nl/bag/i...   

      identificatie  bouwjaar           status gebruiksdoel oppervlakte_min  \
0  0362100001058586      1955  Pand in gebruik  woonfunctie            97.0   
1  0363100012158815      1910  Pand in gebruik  woonfunctie            60.0   
2  0363100012181925      1736  Pand in gebruik  woonfunctie           287.0   
3  0363100012085923      1992  Pand in gebruik  woonfunctie           160.0   
4  0363100012126125      1949  Pand in gebruik  woonfunctie            48.0   

  oppervlakte_max  aantal_verblijfsobjecten  \
0            97.0                         1   
1            83.0     

Turn penalty function

In [15]:
#ANGLE FUNCTION
def calculate_angle(vector1, vector2):
    #calculate dot product and magnitudes
    dot_product = vector1[0] * vector2[2] + vector1[1] * vector2[1]
    mag1 = (vector1[0]**2 + vector1[1]**2) ** 0.5
    mag2 = (vector2[0]**2 + vector2[1]**2) ** 0.5

    #calculate angle in degrees
    angle_rad = np.arcos(dot_product / (mag1 * mag2))
    angle_deg = np.degrees(angle_rad)
    return angle_deg

#TURN WEIGHT FUNCTION
def calculate_turn_weight(u,v,data):
    base_turn_penalty = 30 #based on literature
    max_angle_for_turn = 135 #based on literature

    #Get geom from previous and current edges
    geom = data.get('geometry', None)
    if not geom or len(geom)<2:
        return data.get('length', 0)
    
    #Get nodes' positions
    x1, y1 = G.nodes[u]['x'], G.nodes[u],['y']
    x2, y2 = G.nodes[v]['x'], G.nodes[v],['y']
    vector1 = (x2 - x1, y2 - y1)

    #Identify the node after v to calculate the turn angle
    for next_node in G.neighbors(v):
        if next_node != u: #avoid going back on the path
            x3, y3 = G.nodes[next_node]['x'], G.nodes[next_node]['y']
            vector2 = (x3 - x2, y3 - y2)
            angle = calculate_angle(vector1, vector2)

            if angle <= max_angle_for_turn:
                return data.get('length', 0) + base_turn_penalty
            
    return data.get('length', 0) #no penalty if no turn or no next node


Function to calculate distance between building node and clusters

In [16]:
def get_closest_nodes_gdf(centroid, kd_trees):
    closest_nodes = []
    
    # Convert centroid to a 2D array
    centroid_coord = np.array([[centroid.x, centroid.y]])
    
    # Iterate over all KD-Trees
    for polygon_id, data in kd_trees.items():
        kd_tree = data['kd_tree']
        nodes_in_polygon = data['data']
        
        # Adjust k to the number of points in the KDTree
        num_points = len(nodes_in_polygon)
        if num_points == 0:
            continue  # Skip empty KDTrees
        
        k = min(num_points, 1)  # Query the closest node per KDTree
        
        # Query the KDTree for the closest nodes
        distances, indices = kd_tree.query(centroid_coord, k=k)
        
        # Collect distances and corresponding node IDs
        for distance, index in zip(distances.flatten(), indices.flatten()):
            closest_nodes.append((nodes_in_polygon.index[index], nodes_in_polygon.iloc[index].geometry, distance))
    
    # Create a GeoDataFrame from the closest nodes
    closest_nodes_sorted = sorted(closest_nodes, key=lambda x: x[2])  # Sort by distance
    gdf = gpd.GeoDataFrame(
        closest_nodes_sorted,
        columns=['node_id', 'geometry', 'distance'],
        geometry='geometry',
        crs="EPSG:4326"  # Adjust CRS as needed
    )
    
    return gdf

Calculate walking distance

In [17]:
def calculate_walking_distances(G, source_node, target_nodes_df, weight=calculate_turn_weight, crs='EPSG:4326'):

    # Initialize list to store the shortest path distances
    walking_distances = []
    
    # Calculate the shortest path from source_node to each target node in the graph
    for target_node in target_nodes_df['node_id']:
        try:
            # Compute the shortest path length (walking distance)
            distance = nx.shortest_path_length(G, source=source_node, target=target_node, weight=weight)
        except nx.NetworkXNoPath:
            # If no path exists, set distance to None or a large value
            distance = None
        
        # Append the distance to the list
        walking_distances.append(distance)
    
    # Add the walking distances to the DataFrame
    target_nodes_df['walking_distance'] = walking_distances

    # Convert the DataFrame to GeoDataFrame with the specified CRS
    gdf = gpd.GeoDataFrame(target_nodes_df, geometry='geometry', crs=crs)

    return gdf

Multi threading (original)

In [18]:
# Function to process each building row
def process_building_row(row, kd_trees, G, overlapping_nodes):
    building_centroid = row['centroid']
    source_node = row['closest_node']
    
    # Find the closest nodes for this building centroid
    closest_node_gdf = get_closest_nodes_gdf(building_centroid, kd_trees)
    
    # Sort and take the 5 closest nodes
    closest_node_gdf_sorted = closest_node_gdf.sort_values(by='distance')
    five_closest_nodes = closest_node_gdf_sorted.head(5)

    #exclude nodes that are not within walking distance (more than 2,5 km)
    valid_nodes = five_closest_nodes[five_closest_nodes['distance'] < 2500]

    if valid_nodes.empty:
        return {
            'building_id': row['identificatie'],
            'centroid': building_centroid,
            'source_node': source_node,
            'target_node': None,
            'network_cluster': None,
            'shortest_path': []
        }

    # Calculate walking distances for the 5 closest nodes
    gdf_with_distances = calculate_walking_distances(G, source_node, valid_nodes, weight=calculate_turn_weight)

    closest_node_row = gdf_with_distances.loc[gdf_with_distances['walking_distance'].idxmin()]
    target_node = closest_node_row['node_id']
    network_cluster = overlapping_nodes.loc[int(target_node), 'network_cluster']

    try:
        shortest_path = nx.shortest_path(G, source=source_node, target=target_node, weight=calculate_turn_weight)
    except nx.NetworkXNoPath:
        # If no path exists, set shortest_path to None or an empty list
        shortest_path = []
    
    return {
        'building_id': row['identificatie'],
        'centroid': building_centroid,
        'source_node': source_node,
        'target_node': target_node,
        'network_cluster': network_cluster,
        'shortest_path': shortest_path
    }

# Function to parallelize processing of buildings
def process_buildings_in_parallel(buildings_clean, kd_trees, G, overlapping_nodes):
    num_threads = os.cpu_count()  # Get the number of available threads
    print(f"Using {num_threads} threads for parallel processing")
    
    results = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = {
            executor.submit(process_building_row, row, kd_trees, G, overlapping_nodes): row['identificatie']
            for _, row in buildings_clean.iterrows()
        }
        
        for future in tqdm(concurrent.futures.as_completed(futures), 
                           total=len(futures), 
                           desc="Processing Buildings"):
            results.append(future.result())
    
    return results

# Run function
all_building_distances = process_buildings_in_parallel(buildings_clean, kd_trees, G, overlapping_nodes)

# Convert the results to a DataFrame and export as intermediate output
shortest_paths = pd.DataFrame(all_building_distances)
shortest_paths.to_excel('./root/Outputs/building_shortest_path_to_cluster.xlsx', index=False)

# Display result
display(shortest_paths)


Using 12 threads for parallel processing


Processing Buildings:   0%|          | 0/100 [00:00<?, ?it/s]

,building_id,centroid,source_node,target_node,network_cluster,shortest_path
0,0363100012130093,POINT (116210.27011761811 484145.528855657),814274929,6.378473e+09,54.0,"[814274929, 814274763, 814274928, 4416427359, ..."
1,0363100012165885,POINT (121307.5627702548 485029.54790642526),9462923019,9.462923e+09,30.0,[9462923019]
2,0363100012141142,POINT (124185.21068666207 487053.4393578978),3787095165,9.537326e+09,58.0,"[3787095165, 46383974, 1614187165, 6296435007,..."
3,0384100000001127,POINT (125699.73188401987 482777.64645978477),10977549951,7.353828e+09,59.0,"[10977549951, 7354252382, 10977549949, 1098216..."
4,0363100012104233,POINT (118487.78840366889 484782.4364446469),46290931,6.275410e+09,31.0,"[46290931, 10692138657, 608973421, 6275409963]"
...,...,...,...,...,...,...
95,0363100012164052,POINT (125877.7031054112 488707.9762613534),46451755,4.618019e+09,67.0,"[46451755, 46453301, 46454872, 46455257, 46455..."
96,0384100000011081,POINT (126904.83414103067 484790.1715881264),8933123365,5.530139e+09,25.0,"[8933123365, 8933123364, 8933123360, 893312335..."
97,0394100000219674,POINT (114518.22186702558 483480.77887242846),1077213700,1.213954e+10,38.0,"[1077213700, 1077213698, 779424015, 46244390, ..."
98,0363100012155557,POINT (127749.43837217652 480957.4025857736),1414692826,8.220474e+09,66.0,"[1414692826, 5523446172, 5523444711, 141469282..."
